<font size="4">Creating dataframe of Singaporean districts and their coordinates.</font>

Let us first create a dataframe that contains the districts of Singapore and their coordinates by scraping the data from Wikipedia.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import json # library to handle JSON files

url = "https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore" #Url of Wikipedia page with Toronto neighborhoods and postal codes
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table') #Find tables in the html
planningArea_data = pd.DataFrame(columns=["Planning Area","Latitude","Longitude"])
for row in tables[2].tbody.find_all("tr"):    
    col =row.td
    if col != None:
        planningArea = col.text.strip()
        geolocator = Nominatim(user_agent="sg_explorer")
        address = '{}, SG'.format(planningArea)
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        planningArea_data = planningArea_data.append({"Planning Area": planningArea,"Latitude": latitude,"Longitude" : longitude}, ignore_index=True)
        

Create a map of Singapore and show all the districts.

In [2]:
address = 'Singapore'

#Get the latitude and longitude of Singapore
geolocator = Nominatim(user_agent="sg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude 
longitude = location.longitude

# create map of New York using latitude and longitude values
map_singapore = folium.Map(location=[latitude, longitude], zoom_start=10)

#add markers to map
for lat, lng, neighborhood in zip(planningArea_data["Latitude"], planningArea_data["Longitude"], planningArea_data["Planning Area"]):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_singapore)  
    
map_singapore

Define Foursquare credentials

In [3]:
CLIENT_ID = 'IRKFWJXIIWOMSCV1EBIGSM4C1ODUR1V1WB4XXDKUARYY4WXH' # your Foursquare ID
CLIENT_SECRET = '0B4YEAJQNMY3Y02RUNAWKD0VEX2CD4Q25UH4WPALOSHHNDUU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

<font size="4">Exploring the first district in the planningArea_data Dataframe</font>

In [4]:
print("The first district in the dataframe is {}.".format(planningArea_data.loc[0, "Planning Area"]))

The first district in the dataframe is Ang Mo Kio.


In [5]:
neighborhood_latitude = planningArea_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = planningArea_data.loc[0, 'Longitude'] # neighborhood longitude value

latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '61298bd68791f0302f6ae333'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Ang Mo Kio',
  'headerFullLocation': 'Ang Mo Kio, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 41,
  'suggestedBounds': {'ne': {'lat': 1.3745803045000045,
    'lng': 103.85401568693963},
   'sw': {'lat': 1.3655802954999954, 'lng': 103.84502991306037}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b15f661f964a52012b623e3',
       'name': 'FairPrice Xtra',
       'location': {'address': '#B2-26 AMK Hub',
        'crossStreet': '53 Ang Mo Kio Ave 3',
        'lat': 1.3692792884081397,
        'lng': 103.84888576818767,
        'labeledLat

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories       lat         lng
0  FairPrice Xtra           Supermarket  1.369279  103.848886
1   Old Chang Kee           Snack Place  1.369094  103.848389
2      MOS Burger          Burger Joint  1.369170  103.847831
3             A&W  Fast Food Restaurant  1.369541  103.849043
4          Subway        Sandwich Place  1.369136  103.847612

<font size="4">Exploring the districts of Singapore</font>

First, define a function that can be used to search for venues in each district of Singapore.

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
singapore_venues=getNearbyVenues(list(planningArea_data["Planning Area"]),list(planningArea_data['Latitude']),list(planningArea_data['Longitude']))
print(singapore_venues.shape)
singapore_venues.head()

Ang Mo Kio
Bedok
Bishan
Boon Lay
Bukit Batok
Bukit Merah
Bukit Panjang
Bukit Timah
Central Water Catchment
Changi
Changi Bay
Choa Chu Kang
Clementi
Downtown Core
Geylang
Hougang
Jurong East
Jurong West
Kallang
Lim Chu Kang
Mandai
Marina East
Marina South
Marine Parade
Museum
Newton
North-Eastern Islands
Novena
Orchard
Outram
Pasir Ris
Paya Lebar
Pioneer
Punggol
Queenstown
River Valley
Rochor
Seletar
Sembawang
Sengkang
Serangoon
Simpang
Singapore River
Southern Islands
Straits View
Sungei Kadut
Tampines
Tanglin
Tengah
Toa Payoh
Tuas
Western Islands
Western Water Catchment
Woodlands
Yishun
(1656, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude           Venue  \
0   Ang Mo Kio                1.37008              103.849523  FairPrice Xtra   
1   Ang Mo Kio                1.37008              103.849523   Old Chang Kee   
2   Ang Mo Kio                1.37008              103.849523      MOS Burger   
3   Ang Mo Kio                1.37008              103.849523             A&W   
4   Ang Mo Kio                1.37008              103.849523          Subway   

   Venue Latitude  Venue Longitude        Venue Category  
0        1.369279       103.848886           Supermarket  
1        1.369094       103.848389           Snack Place  
2        1.369170       103.847831          Burger Joint  
3        1.369541       103.849043  Fast Food Restaurant  
4        1.369136       103.847612        Sandwich Place

Continue from Section 2 of lab notebook.